In [12]:
import numpy as np 
import pandas as pd

from gsdmm import MovieGroupProcess

from gensim.models.coherencemodel import CoherenceModel
from gensim import corpora, models
from gensim.utils import simple_preprocess
import gensim
from gensim.models.ldamulticore import LdaMulticore
import re
import spacy

from gensim.models import Phrases
from gensim.models.phrases import Phraser

In [13]:
df=pd.read_csv('sdg_preprocessed.csv')

In [14]:
df

,Unnamed: 0,date,Tweet,labels
0,0,2021-10-15 00:05:27+00:00,çinde den e kadar asgari ücret kodlaması var ...,negative
1,1,2021-10-15 00:11:08+00:00,aman aman önemli işler yapıyorsunuzeytasgari ü...,negative
2,2,2021-10-15 00:22:51+00:00,asgari ücret yıla bedel olacak,negative
3,3,2021-10-15 00:23:21+00:00,asgari ücret,negative
4,4,2021-10-15 00:31:42+00:00,dolar olmuş ülkede tek zam gelmeyen asgari üc...,negative
...,...,...,...,...
749182,749182,2023-02-12 22:57:51+00:00,NaN,negative
749183,749183,2023-02-12 23:04:45+00:00,ekonomiyi çökerttiler derken ak parti öncesind...,negative
749184,749184,2023-02-12 23:13:45+00:00,yandaş medya el ele vermiş emekliye gün doğdu ...,negative
749185,749185,2023-02-12 23:21:58+00:00,özel okul öğretmenlerinin asgari ücret altına ...,negative


In [15]:
positive = df[df["labels"] == "positive"]

In [16]:
positive

,Unnamed: 0,date,Tweet,labels
10,10,2021-10-15 02:00:00+00:00,türki̇ş başkanından asgari ücret zammıyla ilgi...,positive
12,12,2021-10-15 02:22:07+00:00,dolar tlenflasyon asgari ücret tl sizeoyumu...,positive
27,27,2021-10-15 04:47:59+00:00,oyun basitüç çocuk öğrenci marketpazarfaturala...,positive
30,30,2021-10-15 04:57:47+00:00,sabah vardiyasına giden gültepe mahallesindeki...,positive
33,33,2021-10-15 05:02:22+00:00,asgari ücret zam artmaz üretim artar insan hak...,positive
...,...,...,...,...
749144,749144,2023-02-12 19:42:31+00:00,i̇lk günden güne sanatçı imkanı olan herkese ç...,positive
749151,749151,2023-02-12 20:10:09+00:00,belediyeler inşaat denetimlerine giden mühendi...,positive
749156,749156,2023-02-12 20:38:35+00:00,hatırlamazsınız ben ’lerimin başlarındaydım bu...,positive
749158,749158,2023-02-12 20:55:50+00:00,milyonlarca kişinin maaşı değişiyor yeni asgar...,positive


In [17]:
def make_n_grams(texts):
    bigram = gensim.models.Phrases(texts, min_count=5, threshold=100)  # higher threshold fewer phrases.
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram = gensim.models.Phrases(bigram[texts], threshold=100)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    bigrams_text = [bigram_mod[doc] for doc in texts]
    trigrams_text =  [trigram_mod[bigram_mod[doc]] for doc in bigrams_text]
    return trigrams_text

In [18]:
positive['Tweet'] = positive.Tweet.values.tolist()
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence)))

C:\Users\Alkan\AppData\Local\Temp\ipykernel_16392\2840482353.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive['Tweet'] = positive.Tweet.values.tolist()


In [19]:
tokens_tweets = list(sent_to_words(positive['Tweet']))

In [20]:
np.random.seed(0)

In [21]:
mgp = MovieGroupProcess(K=10, alpha=0.01, beta=0.01, n_iters=30) 

vocab = set(x for tweet in tokens_tweets for x in tweet)
n_terms = len(vocab)
model = mgp.fit(tokens_tweets, n_terms)

In stage 0: transferred 158115 clusters with 10 clusters populated
In stage 1: transferred 107250 clusters with 10 clusters populated
In stage 2: transferred 59782 clusters with 10 clusters populated
In stage 3: transferred 42127 clusters with 10 clusters populated
In stage 4: transferred 34913 clusters with 10 clusters populated
In stage 5: transferred 31441 clusters with 10 clusters populated
In stage 6: transferred 29571 clusters with 10 clusters populated
In stage 7: transferred 28713 clusters with 10 clusters populated
In stage 8: transferred 27222 clusters with 10 clusters populated
In stage 9: transferred 26230 clusters with 10 clusters populated
In stage 10: transferred 25206 clusters with 10 clusters populated
In stage 11: transferred 24644 clusters with 10 clusters populated
In stage 12: transferred 24508 clusters with 10 clusters populated
In stage 13: transferred 24051 clusters with 10 clusters populated
In stage 14: transferred 23624 clusters with 10 clusters populated
In 

In [22]:
def top_words(cluster_word_distribution, top_cluster, values):
    for cluster in top_cluster:
        sort_dicts =sorted(mgp.cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True)[:values]
        print("\nCluster %s : %s"%(cluster,sort_dicts))

In [23]:
doc_count = np.array(mgp.cluster_doc_count)
print('Number of documents per topic :', doc_count)

# topics sorted by the number of document they are allocated to
top_index = doc_count.argsort()[-15:][::-1]
print('\nMost important clusters (by number of docs inside):', top_index)
# show the top 5 words in term frequency for each cluster 
top_words(mgp.cluster_word_distribution, top_index, 20)

Number of documents per topic : [36000  9549 24434 29160 27131 20466 20373  8484 10592  5982]

Most important clusters (by number of docs inside): [0 3 4 2 5 6 8 1 7 9]

Cluster 0 : [('asgari', 35353), ('ücret', 33797), ('bir', 9307), ('tl', 5578), ('kadar', 3955), ('var', 3108), ('değil', 3031), ('yok', 2829), ('bin', 2771), ('olsun', 2728), ('mi', 2514), ('maaş', 2471), ('asgariücret', 2312), ('ben', 2136), ('zam', 2104), ('bile', 1910), ('maaşı', 1903), ('emekli', 1780), ('dolar', 1636), ('lira', 1607)]

Cluster 3 : [('asgari', 32970), ('ücret', 30061), ('açıklaması', 8077), ('bakan', 5061), ('erdoğan', 4265), ('cumhurbaşkanı', 4004), ('tespit', 3992), ('türki', 3937), ('komisyonu', 3680), ('bilgin', 3632), ('bir', 3224), ('eyt', 3166), ('çalışma', 3163), ('son', 2890), ('bilginden', 2780), ('bugün', 2476), ('sosyal', 2374), ('bakanı', 2356), ('güvenlik', 2326), ('asgariücret', 2115)]

Cluster 4 : [('asgari', 27258), ('ücret', 25812), ('tl', 6559), ('bir', 6120), ('zam', 5339), ('em